In [1]:
from src.mitigation.FairContrastiveLearning import FairContrastiveLearningBenchmark
from src.fairness_libraries.fair_contrastive_learning.FairSupConModels import FairSupConResNet18, LinearClassifier
from src.mitigation.FairContrastiveLearning import TwoCropTransform

In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [5]:
from torchvision import transforms
train_transform = transforms.Compose([
        transforms.RandomResizedCrop(size=(128,128), scale=(0.2, 1.)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomApply([
            transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)
        ], p=0.8),
        transforms.RandomGrayscale(p=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5000, 0.5000, 0.5000), std=(0.5000, 0.5000, 0.5000))
    ])

In [ ]:
%reload_ext autoreload
from src.datasets.CelebA_dataset import CelebADataset

celeba_contrastive_dataset = CelebADataset(input_shape=(128,128), usage=0.1, transform=TwoCropTransform(train_transform))
celeba_classifier_dataset = CelebADataset(input_shape=(128,128), usage=0.1)

dataset_class = type(celeba_classifier_dataset)

target_name, protected_feature_name = celeba_classifier_dataset.get_target_protected_feature_names()
celeba_contrastive_dataset = dataset_class(input_shape=(128, 128), transform=TwoCropTransform(train_transform), target_name=target_name, protected_feature_name=protected_feature_name)


In [5]:
%reload_ext autoreload
celeba_contrastive_model = FairSupConResNet18()
celeba_classifier = LinearClassifier(num_classes=celeba_classifier_dataset.get_num_classes_target())

fair_contrastive_benchmark_celeba = FairContrastiveLearningBenchmark(celeba_contrastive_model, celeba_classifier, celeba_contrastive_dataset, celeba_classifier_dataset, device="mps", model_path="fair_contrastive/celeba/default_test", batch_size=32)
fair_contrastive_benchmark_celeba.run_benchmark()

In [ ]:
from src.datasets.UTKFaceDataset import UTKFaceDataset
%reload_ext autoreload
utk_contrastive_dataset = UTKFaceDataset(input_shape=(128,128), transform=TwoCropTransform(train_transform))
utk_classifier_dataset = UTKFaceDataset(input_shape=(128,128))

utk_contrastive_model = FairSupConResNet18()
utk_classifier = LinearClassifier(num_classes=utk_classifier_dataset.get_num_classes_target())

fair_contrastive_benchmark_utk = FairContrastiveLearningBenchmark(utk_contrastive_model, utk_classifier, utk_contrastive_dataset, utk_classifier_dataset, device="cuda", model_path="fair_contrastive/utk_face/11052024", batch_size=512)
fair_contrastive_benchmark_utk.run_benchmark()